# Fast blocked Gibbs sampling

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QAHRoddur/JWAS/blob/main/Examples/fast_blocked_gibbs_sampling.ipynb)

This notebook is auto-generated from the JWAS.jl wiki page.


In [ ]:
using Pkg
Pkg.add("JWAS")
Pkg.precompile()
using JWAS


#  Fast blocked Gibbs sampling
The fast blocked Gibbs sampling for Bayesian linear mixed model is proposed in below papers: 
> Breen, E.J., MacLeod, I.M., Ho, P.N. et al. BayesR3 enables fast MCMC blocked processing for largescale multi-trait genomic prediction and QTN mapping analysis. Commun Biol 5, 661 (2022). https://doi.org/10.1038/s42003-022-03624-1

> Zhao, T., Fernando, R., Garrick, D. et al. Fast parallelized sampling of Bayesian regression models for whole-genome prediction. Genet Sel Evol 52, 16 (2020). https://doi.org/10.1186/s12711-020-00533-x



# JWAS
The fast block mode can be activated via the `fast_blocks` argument in the `runMCMC()` function. The `fast_blocks` argument can be assigned the number of SNPs in each block (i.e., block size). For example, for 50k SNP, when `chain_length=1000` and `fast_blocks=100`, there will be 500 blocks, each containing 100 SNPs (i.e., block size=100 SNPs). If `fast_blocks=true`, then block size is the square root of the number of individuals. Note that when the fast_blocks mode is enabled, the number of SNPs in each block will be displayed in JWAS printouts, e.g., "BLOCK SIZE: 100".

**JWAS code to determine block size**:
```{julia}
if fast_blocks == true
    block_size = Int(floor(sqrt(mme.M[1].nObs)))
elseif typeof(fast_blocks) <: Number
    block_size = Int(floor(fast_blocks))
end


In [ ]:
**JWAS structure for outer and inner iterations**:

#total iterations = #outer iterations $*$ #inner iterations

```{julia}
#MCMC iteration start
for k in 1:outer_niter
    for i in 1:blocks
        nreps = block_size #nreps=block_size, outer_niter=niter/block_size
        for reps = 1:nreps
            for j=1:block_size
                #sample marker effects
	    end
	end
    end 
end
#MCMC iteration end


*  for example: 50k SNPs, block size = 100 SNPs (thus 500 blocks), 2000 total iterations
* 2000 total iterations = 20  outer iterations  * 100 inner iterations
* in each outer iteration:  collect 100 MCMC samples for 100 SNPs in block1,......, collect 100 MCMC samples for 100 SNPs in block500 --> in total, we collect 100 MCMC samples for 50k SNPs.
* in each outer iteration, we have 100 MCMC samples for all SNPs. --> Thus, for all 20 outer iterations, we will have 2000 MCMC samples for all SNPs.


# JWAS examples
Note that `fast_blocks` now only works for one class of genotype, so `fast_blocks` cannot be used for [Multi-class Bayesian Analysis](https://github.com/reworkhow/JWAS.jl/wiki/Multiclass-Bayesian-analysis).

## Univariate Linear Mixed Model (`fast_blocks`)
```{julia}
# Step 1: Load packages
using JWAS,DataFrames,CSV,Statistics,JWAS.Datasets, Random
Random.seed!(123)

# Step 2: Read data
phenofile = dataset("phenotypes.txt", dataset_name="demo_7animals")
pedfile = dataset("pedigree.txt", dataset_name="demo_7animals")
genofile = dataset("genotypes.txt", dataset_name="demo_7animals")
phenotypes = CSV.read(phenofile,DataFrame,delim = ',',header=true,missingstring=["NA"])
pedigree = get_pedigree(pedfile,separator=",",header=true);
genotypes = get_genotypes(genofile,separator=',',method="BayesC");
first(phenotypes,5)

# Step 3: Build Model Equations
model_equation ="y1 = intercept + x1 + x2 + x2*x3 + ID + dam + genotypes";
model = build_model(model_equation);

# Step 4: Set Factors or Covariates
set_covariate(model,"x1");

# Step 5: Set Random or Fixed Effects
set_random(model,"x2");
set_random(model,"ID dam",pedigree);

# Step 6: Run Analysis
out=runMCMC(model,phenotypes,chain_length = 1000, fast_blocks = 100);

# Step 7: Check Results
out["EBV_y1"]  # estimated breeding values


In [ ]:
## Single-trait Single-step Analysis (`fast_blocks`)
```{julia}
# Step 1: Load packages
using JWAS,DataFrames,CSV,Statistics,JWAS.Datasets, Random
Random.seed!(123)

# Step 2: Read data
phenofile = "../data/phenotypes.txt"
pedfile = "../data/pedigree.txt"
genofile = "../data/genotypes.txt"
phenotypes = CSV.read(phenofile,DataFrame,delim = ',',header=true,missingstring=["NA"])
pedigree = get_pedigree(pedfile,separator=",",header=true);
genotypes = get_genotypes(genofile,separator=',',method="BayesC");
first(phenotypes,5)

# Step 3: Build Model Equations
model_equation ="y1 = intercept + x1 + x2 + x2*x3 + ID + dam + genotypes";
model = build_model(model_equation);

# Step 4: Set Factors or Covariates
set_covariate(model,"x1");

# Step 5: Set Random or Fixed Effects
set_random(model,"x2");
set_random(model,"ID dam",pedigree);

# Step 6: Run Analysis
out=runMCMC(model,phenotypes,single_step_analysis=true,pedigree=pedigree,chain_length = 1000, fast_blocks = 100);

# Step 7: Check Results
out["EBV_y1"]  # estimated breeding values


## Multivariate Linear Mixed Model (`fast_blocks`)
```{julia}
# Step 1: Load packages
using JWAS,DataFrames,CSV,Statistics,JWAS.Datasets, Random
Random.seed!(123)

# Step 2: Read data
phenofile =  dataset("phenotypes.txt", dataset_name="demo_7animals")
pedfile =  dataset("pedigree.txt", dataset_name="demo_7animals")
genofile =  dataset("genotypes.txt", dataset_name="demo_7animals")
phenotypes = CSV.read(phenofile,DataFrame,delim =  ',',header=true,missingstring=["NA"])
pedigree =  get_pedigree(pedfile,separator=",",header=true);
genotypes =  get_genotypes(genofile,separator=',',method="BayesC");
first(phenotypes,5)

# Step 3: Build Model Equations
model_equation ="y1 = intercept + x1 + x2 + x2*x3 + ID + dam + genotypes
                 y2 = intercept + x1 + x2 + ID + genotypes
                 y3 = intercept + x1 + ID + genotypes";
model =  build_model(model_equation);

# Step 4: Set Factors or Covariates
set_covariate(model,"x1");

# Step 5: Set Random or Fixed Effects
set_random(model,"x2");
set_random(model,"ID dam",pedigree);

# Step 6: Run Analysis
out=runMCMC(model,phenotypes,chain_length =  1000, fast_blocks =  100);

# Step 7: Check Results
out["EBV_y3"]  # estimated breeding values
